In [0]:
%fs ls dbfs:/user/hive/warehouse/authentication_credentials



path,name,size,modificationTime
dbfs:/user/hive/warehouse/authentication_credentials/5u/,5u/,0,1723032357336
dbfs:/user/hive/warehouse/authentication_credentials/_delta_log/,_delta_log/,0,1723032357336


In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import * # pyspark functions
import urllib # URL processing

# set auth credentials from 'Delta table'
delta_table_path = "dbfs:/user/hive/warehouse/authentication_credentials"
keys_df = spark.read.format("delta").load(delta_table_path)

# Get the AWS access key and secret key from the spark dataframe
ACCESS_KEY = keys_df.select('Access key ID').collect()[0]['Access key ID']
SECRET_KEY = keys_df.select('Secret access key').collect()[0]['Secret access key']
# Encode the secrete key
ENCODED_SECRET_KEY = urllib.parse.quote(string=SECRET_KEY, safe="")


In [0]:
AWS_S3_BUCKET = "user-126ca3664fbb-bucket"
MOUNT_NAME = "/mnt/126ca3664fbb-mount"
SOURCE_URL = "s3n://{0}:{1}@{2}".format(ACCESS_KEY, ENCODED_SECRET_KEY, AWS_S3_BUCKET)

dbutils.fs.mount(SOURCE_URL, MOUNT_NAME)

Out[43]: True

In [0]:
display(dbutils.fs.ls("/mnt/126ca3664fbb-mount/topics/126ca3664fbb.geo/"))
# to unmount
# dbutils.fs.unmount("/mnt/<mount name>")


path,name,size,modificationTime
dbfs:/mnt/126ca3664fbb-mount/topics/126ca3664fbb.geo/partition=0/,partition=0/,0,1723036007314


In [0]:
%sql
--Disable format checks during the reading of Delta tables
SET spark.databricks.delta.formatCheck.enabled=false

key,value
spark.databricks.delta.formatCheck.enabled,false


In [0]:
file_location = "/mnt/126ca3664fbb-mount/topics/126ca3664fbb.{}/partition=0/*.json"
file_type = "json"
# Ask Spark to infer the schema
infer_schema = "true"
# Read in JSONs from mounted S3 bucket
df_pin = spark.read.format(file_type) \
.option("inferSchema", infer_schema) \
.load(file_location.format("pin"))

df_geo = spark.read.format(file_type) \
.option("inferSchema", infer_schema) \
.load(file_location.format("geo"))

df_user = spark.read.format(file_type) \
.option("inferSchema", infer_schema) \
.load(file_location.format("user"))

display(df_geo)

country,ind,latitude,longitude,timestamp
Antarctica (the territory South of 60 deg S),2418,-88.4642,-171.061,2022-05-27T11:30:59
Cocos (Keeling) Islands,10794,-89.5236,-154.567,2022-01-01T02:26:50
Cocos (Keeling) Islands,10794,-89.5236,-154.567,2022-01-01T02:26:50
Central African Republic,2074,-52.3213,-50.11,2019-11-03T05:41:59
Dominican Republic,9979,14.9967,-120.682,2018-07-18T19:01:46
Cote d'Ivoire,2923,-84.6302,-164.507,2019-09-08T22:53:09
Cote d'Ivoire,2923,-84.6302,-164.507,2019-09-08T22:53:09
French Guiana,8304,-28.8852,-164.87,2019-09-13T04:50:29
Cote d'Ivoire,4315,-45.8508,66.1003,2019-12-15T03:51:28
French Guiana,8304,-28.8852,-164.87,2019-09-13T04:50:29
